## RFP to Proposal Project

Import Libraries

In [21]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI
from dotenv import load_dotenv
import re 
import time
from copy import deepcopy
from PyPDF2 import PdfReader
from docx import Document
import openai
import json
import ast
import pandas as pd

# 최신 LangChain 기준
from langchain_core.documents import Document
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
import gradio as gr

Parameters

In [2]:
# Load environment variables in a file called .env

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
serp_api_key = os.getenv("SERP_API_KEY")
perplexity_api_key = os.getenv("PEPLEXITY_API_KEY")
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

if serp_api_key:
    print(f"serp_api_key exists and begins {serp_api_key[:8]}")
else:
    print("serp_api_key not set")

if perplexity_api_key:
    print(f"perplexity_api_key exists and begins {perplexity_api_key[:8]}")
else:
    print("perplexity_api_key not set")

# GPT 모델 선언
openai = OpenAI()
MODEL = 'gpt-4o'

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyBU
serp_api_key exists and begins c3ee9cec
perplexity_api_key exists and begins pplx-r4f


In [3]:
# ppt template으로 실제 제안서에 사용될 정도의 quality가 나올수 있도록 고도화 필요
# 같은 슬라이드라도 slide_description에 따라 제안서 mapping 필요 
# ex). requirement_detail_v1, requirement_detail_v2

PROPOSAL_SLIDE_TEMPLATES = {
    "cover_page": {
        "elements": {
            "Title": "프로젝트의 정식 명칭 (중앙 상단에 크게 배치) (예시)",
            "Subtitle": "고객사명 또는 부제 설명 (Title 아래 위치) (예시)",
            "ProjectDate": "제안서 작성 또는 제출 일자 (하단 우측 또는 좌측 구석에 배치) (예시)",
            "PreparedBy": "작성자 또는 제안 주체 (ProjectDate 인근 또는 하단 중앙) (예시)",
            "Logo": "회사 또는 고객사 로고 (우상단 또는 좌상단에 적절히 배치) (예시)"
        },
    },
    "table_of_contents": {
        "slide_description": "전체 제안서의 슬라이드 구성을 한눈에 파악할 수 있도록 시각적으로 정리합니다. (예시)",
        "description": "",
        "elements": {
            "Title": "목차 제목 (상단 중앙) (예시)",
            "SectionList": "슬라이드별 주요 제목 리스트 (Bullet 형식으로 왼쪽 정렬) (예시)"
        },
    },
    "executive_summary": {
        "slide_description": "제안서 전체의 핵심 내용을 1~2페이지 내에 요약하여 임원 또는 의사결정자가 빠르게 이해할 수 있도록 구성합니다. (예시)",
        "description": "",
        "elements": {
            "Title": "슬라이드 제목 (예: 제안 개요, Executive Summary) (예시)",
            "MiddleText": "프로젝트 요약 또는 핵심 문장 (제목 아래 강조 박스) (예시)",
            "SummaryPoints": "핵심 제안 내용 Bullet (전략, 기대 효과, 기간, 투자 규모 등) (예시)",
            "ClientValuePoints": "SummaryPoints 별 고객에게 제공되는 핵심 가치 또는 차별화된 이점 요약 (예시)",
            "QuoteBox": "고객사 경영진의 인용문 또는 고객의 핵심 니즈를 대변하는 문장 (슬라이드 하단 강조용) (예시)",
            "MetricsOverview": "정량 지표 요약 박스 (예: RFP의 KPI 등) (예시)"
        },
        "needs_research": []
    },
    "project_understanding": {
        "slide_description": "프로젝트의 필요성과 배경을 설명하며, 고객의 상황과 과제를 명확히 인식하고 있다는 점을 전달합니다. (예시)",
        "description": "",
        "elements": {
            "Title": "슬라이드 제목 (상단 중앙) (예시)",
            "MiddleText": "프로젝트 요약 또는 핵심 문장 (제목 아래 강조 박스) (예시)",
            "KeyObjectives": "고객의 주요 목표 목록 (왼쪽 열에 Bullet 형식) (예시)",
            "BackgroundIssues": "해결하고자 하는 문제 또는 현재 상황 (오른쪽 열에 Bullet 형식) (예시)",
            "StrategicImplications": "현 상황이 비즈니스에 미치는 영향 (예: 고객경험 저하, 비용 상승 등) (예시)"
        },
        "needs_research": []
    },
    "client_needs_summary": {
        "slide_description": "고객의 구체적인 요구사항을 명확하게 정리해 실무적 방향성과 대응의 기준을 제시합니다. (예시)",
        "description": "",
        "elements": {
            "Title": "슬라이드 제목 (상단 중앙) (예시)",
            "MiddleText": "요구사항 요약 핵심 문장 (예시)",
            "NeedsBulletPoints": "고객 니즈 요약 문장 리스트 (예시)",
            "StrategicDrivers": "고객 요구를 유발한 내부/외부 전략 요인 (ex. 정책 변화, DX 전략 등)",
            "NeedsMatrix": "요구사항을 정리한 표 (예시)",
            "ImplicationColumn": "NeedsMatrix 내 각 요구에 따른 비즈니스적 영향 또는 구현 난이도 칼럼 추가 (예시)",
            "RFPQuoteHighlights": "RFP 문서 내 직접적인 인용문 (ex. 운영 효율성을 20% 이상 개선해야 함) 필요시"
        },
        "needs_research": []
    },
    "requirement_detail": {
        "slide_description": "프로젝트 또는 제안의 성공을 위해 반드시 충족되어야 할 요구사항(client_needs_summary의 NeedsBulletPoints)을 각 항목별로 명확하게 설명하는 슬라이드입니다. 각 요구사항의 정의, 구체적 설명, 우선순위, 그리고 관련된 예시나 수용 기준을 포함해 실무적 방향성을 제시합니다.",
        "description": "",
        "elements": {
            "Title": "슬라이드 제목 (상단 중앙) (예시)",
            "MiddleText": "요구사항 전체를 아우르는 요약 문장 또는 슬라이드의 핵심 목적 (예시)",
            "RequirementDescription": "한가지 요구사항에 대한 상세 설명 (예시, 각 Bullet 옆에 간략히 설명 또는 하단에 별도 박스 형태로 추가)",
            "ResponsePlanBox" : "각 요구사항에 대해 우리 측이 어떻게 대응하고 충족할 것인지 요약적으로 제시(필수)",
            "AcceptanceCriteria": "각 요구사항이 충족되었음을 판단할 수 있는 구체적 기준 또는 예시 (예: 테스트 통과 조건, 성능 수치 등)",
            "CategoryTag": "요구사항의 유형 분류 태그 (예: 기능적, 비기능적, 비즈니스, 기술 등)",
            "DependencyNote": "해당 요구사항과 연관된 다른 요구사항 또는 선행 조건에 대한 설명 (필요시)"
        },
        "needs_research": []
    },
    "market_analysis_market_overview": {
        "slide_description": "해당 산업의 크기, 구조, 주요 트렌드를 데이터와 함께 제시해 전체 시장 배경을 설명합니다. (예시)",
        "description": "",
        "elements": {
            "Title": "슬라이드 제목 (예시)",
            "MiddleText": "시장 개요 요약 문장 (예시)",
            "GraphLeft": "시장 관련 그래프 (예: 점유율, 성장률 등) (예시)",
            "TextRight": "시장 현황 및 주요 특징 설명 텍스트 (예시)",
            "MarketSegmentationTable": "시장 세분화 표 (예: 고객 유형, 지역, 제품군 등) (예시)",
            "KeyPlayersBox": "주요 기업 리스트 및 시장 내 포지셔닝 설명 (예시)",
            "InsightBox": "시장 전체에서 가장 주목할 트렌드 1~2가지 강조 인사이트 (아이콘/강조 색 박스) (예시)"
        },
        "needs_research": []
    },
    "growth_trend_analysis": {
        "slide_description": "시장 혹은 기술의 성장세를 데이터 기반으로 시각화하고 그 시사점을 제시합니다. (예시)",
        "description": "",
        "elements": {
            "Title": "슬라이드 제목 (예시)",
            "MiddleText": "성장 트렌드 핵심 요약 문장 (예시)",
            "GraphLeft": "성장 추이 그래프 (선형, 누적 등) (예시)",
            "TextRight": "그래프 해석 및 성장 요인 서술 (예시)",
            "GrowthDrivers": "성장 동력 요약 리스트 (예: 기술 혁신, 정책 수혜 등) (예시)",
            "ForecastBox": "향후 전망 수치 (예: CAGR, 2028년까지 3배 성장 등) 강조 박스 (예시)",
            "BenchmarkComparison": "동일 산업 내 경쟁국 또는 글로벌 평균과 비교 분석 그래프/표 (예시)",
            "OpportunityTags": "고성장 영역에 대한 기회 태그 (예: GenAI, SaaS, Micro-mobility 등) (예시)"
        },
        "needs_research": []
    },
    "industry_drivers_challenges": {
        "slide_description": "산업 내 성장 요인과 저해 요인을 대조해 전략 방향 설정의 기초로 삼습니다. (예시)",
        "description": "",
        "elements": {
            "Title": "슬라이드 제목 (예시)",
            "MiddleText": "산업 변화 요약 또는 주요 시사점 (예시)",
            "DriversList": "산업 동인 목록 (왼쪽 컬럼 Bullet 형식) (예시)",
            "ChallengesList": "산업 장애 요소 목록 (오른쪽 컬럼 Bullet 형식) (예시)",
            "DetailComments": "각 항목에 대한 보충 설명 또는 전략적 인사이트 (예시)",
            "StrategicImplicationsBox": "전략 수립 시 반드시 고려해야 할 핵심 인사이트 정리 (박스 형태) (예시)",
            "QuoteOrDataSupport": "각 요인에 대한 정량적 근거 또는 고객/전문가 인용문 (예시)"
        },
        "needs_research": []
    },
    "competitive_benchmarking": {
        "slide_description": "경쟁사 대비 자사의 상대적 위치와 우위를 정량적·정성적으로 설명합니다. (예시)",
        "description": "",
        "elements": {
            "Title": "슬라이드 제목 (예시)",
            "MiddleText": "경쟁 포지셔닝 요약 (예시)",
            "TableMain": {
                "description": "핵심 비교 지표 (예: 가격, 기능 수, 고객 수 등) (예시)",
                "table_type": "하이라이트 테이블 (예시)",
                "highlight_column": "자사 기준 (예시)",
                "data_table": [["항목", "자사", "경쟁사(KPMG)", "경쟁사(딜로이트)"], ["기능 수", "12", "10", "9"]]
            },
            "BenchmarkTable": {
                "description": "세부 항목 비교 (세부 제품/서비스 기준) (예시)",
                "table_type": "Matrix 테이블 (예시)",
                "format": "행: 기업 / 열: 항목 (예: UX, 확장성, 기술 스택 등) (예시)"
            },
            "CompetitiveInsights": "경쟁사 전략 및 고객 시사점 도출 (예시)",
            "TextBottom": "요약 또는 인사이트 도출 문장 (예시)",
            "DifferentiationBox": "자사의 차별화 요소를 강조하는 요약 박스 (예: Only, First, Unique 등) (예시)",
            "CustomerPerception": "시장 내 고객의 인지도 또는 만족도 관련 데이터 시각화 (설문 or NPS) (예시)"
        },
        "needs_research": []
    },
    "swot_analysis": {
        "slide_description": "내부·외부 환경 분석을 통해 전략적 포지션을 진단하고 방향성을 제시합니다. (예시)",
        "description": "",
        "elements": {
            "Title": "슬라이드 제목 (예시)",
            "MiddleText": "SWOT 분석 요약 문장 (예시)",
            "SWOT-S": "Strength 요소 리스트 (예시)",
            "SWOT-W": "Weakness 요소 리스트 (예시)",
            "SWOT-O": "Opportunity 요소 리스트 (예시)",
            "SWOT-T": "Threat 요소 리스트 (예시)",
            "NarrativeSummary": "요약 설명 또는 전략적 시사점 (예시)",
            "TOWSMatrix": "SWOT 기반 전략 매트릭스 (예: SO, ST, WO, WT 전략 각각 예시) (예시)",
            "ImpactAssessmentTable": "각 요소의 영향도 및 긴급도 평가 (High/Medium/Low) (예시)"
        },
        "needs_research": []
    },
    "technical_specifications": {
        "slide_description": "제안 솔루션의 기술적 구조, 사용 기술, 플랫폼 등을 상세히 설명합니다. (예시)",
        "description": "",
        "elements": {
            "Title": "슬라이드 제목 (예시)",
            "MiddleText": "기술 사양 요약 (예시)",
            "TechStackDiagram": "기술 스택 구조도 (Front-End, Back-End, DB 등) (예시)",
            "SystemCompatibility": "플랫폼/브라우저 호환성 설명 (예시)",
            "SecurityFeatures": "보안 및 인증 체계 요약 (예시)",
            "PerformanceMetrics": "성능 지표 (예시)",
            "IntegrationPoints": "외부 시스템과의 연동 포인트 및 방식 (API/FTP 등) (예시)",
            "TechRiskMitigation": "기술 리스크 및 대응 방안 (예시)",
            "VersioningPolicy": "버전 관리 정책 및 릴리즈 주기 (예시)"
        },
        "needs_research": []
    },
    "service_operation_model": {
        "slide_description": "서비스 운영 및 유지보수 체계를 설명하여 안정성 및 지속 가능성을 강조합니다. (예시)",
        "description": "",
        "elements": {
            "Title": "슬라이드 제목 (예시)",
            "MiddleText": "운영 모델 요약 (예시)",
            "OperationOrgChart": "운영 조직도 또는 지원 체계도 (예시)",
            "MaintenancePlan": "유지보수 및 정기점검 계획 (예시)",
            "SupportChannels": "지원 채널 및 운영 시간 정보 (예시)",
            "CustomerFeedbackLoop": "고객 피드백 수집 및 반영 절차 (예시)",
            "MonitoringTools": "사용 중인 모니터링/알림 시스템 (예: Grafana, Slack Alert 등) (예시)"
        },
        "needs_research": []
    },
    "compliance_and_governance": {
        "slide_description": "법적/정책적 준수 사항 및 거버넌스 체계를 설명합니다. (예시)",
        "description": "",
        "elements": {
            "Title": "슬라이드 제목 (예시)",
            "MiddleText": "컴플라이언스 및 거버넌스 요약 (예시)",
            "ComplianceList": "준수 기준 리스트 (예시)",
            "PolicyDiagram": "보안/정책 흐름도 (예시)",
            "DataProtectionMeasures": "데이터 보호 및 접근제어 조치 (예시)",
            "AuditAndMonitoring": "감사 및 모니터링 체계 요약 (예시)",
            "RiskResponseMatrix": "위협/리스크 발생 시 대응 전략 매트릭스 (예: 탐지 → 대응 → 복구) (예시)",
            "GovernanceRoles": "거버넌스 참여 부서 및 R&R 요약 테이블 (예시)",
            "RealTimeMonitoringTools": "SIEM, DLP 등 활용 중인 실시간 모니터링 시스템 요약 (예시)"
        },
        "needs_research": []
    },
    "client_case_references": {
        "slide_description": "과거 수행한 유사 프로젝트 사례를 통해 신뢰성과 경험을 강조합니다. (예시)",
        "description": "",
        "elements": {
            "Title": "슬라이드 제목 (예시)",
            "MiddleText": "사례 요약 (예시)",
            "CaseList": "유사 프로젝트 리스트 (고객사, 수행 내용, 성과 등) (예시)",
            "VisualHighlights": "대표 사례 이미지 또는 그래프 (예시)",
            "SuccessIndicators": "주요 성과 지표 요약 (예시)",
            "ClientTestimonials": "고객사의 평가 또는 인용문 (예시)",
            "IndustryDiversityGraph": "수행 경험 산업군 분포 (도넛 차트 등) (예시)",
            "CaseImpactSummary": "각 사례의 전략적 효과 요약 (예: 고객만족도 증가, 보안 리스크 감소 등) (예시)"
        },
        "needs_research": []
    },
    "solution_overview": {
        "slide_description": "제안 솔루션의 핵심 가치와 구조를 시각화하고 간명하게 전달합니다. (예시)",
        "description": "",
        "elements": {
            "Title": "슬라이드 제목 (예시)",
            "ValuePropositionBox": {
                "description": "고객에게 제공되는 3대 핵심 가치 또는 효익 (ex: 비용 절감, UX 개선, 확장성) (예시)",
                "format": ["가치 1 (예시)", "가치 2 (예시)", "가치 3 (예시)"]
            },
            "SolutionCategoryTag": "해당 솔루션이 속한 유형 (예: SaaS / 온프레미스 / 하이브리드 / 컨설팅 / 플랫폼 등) (예시)",
            "SimpleDiagram": {
                "description": "End-to-End 흐름 또는 Value Chain 시각화 (예시)",
                "visual_type": "Linear / Circular / 3단계 프로세스 등 (예시)"
            },
            "ArchitectureDiagram": {
                "description": "기술/기능 구조 시각화 (예시)",
                "visual_type": "Layer 구조도, Block Diagram, 모듈 구성도 등 (예시)"
            },
            "KeyModules": {
                "description": "주요 모듈 또는 기능 구성 요소 리스트 (각 항목에 간단한 설명 포함) (예시)",
                "format": [
                    {"Module": "Data Collector (예시)", "Function": "실시간 데이터 수집 (예시)"},
                    {"Module": "AI Analyzer (예시)", "Function": "패턴 분석 및 이상 탐지 (예시)"},
                    {"Module": "Dashboard (예시)", "Function": "시각화 및 리포팅 (예시)"}
                ]
            },
            "FlexibleIntegrationPoints": {
                "description": "다양한 외부 시스템과 연동 가능한 인터페이스 또는 API 설명 (예시)",
                "examples": ["ERP (예시)", "CRM (예시)", "Open API (예시)", "클라우드 스토리지 등 (예시)"]
            },
            "ScalabilityAndSecurity": {
                "description": "확장성 및 보안 체계 설명 (예시)",
                "points": ["모듈 확장 가능 구조 (예시)", "RBAC 기반 접근 통제 (예시)", "암호화 저장 (예시)"]
            }
        },
        "needs_research": []
    },
    "use_case_scenarios": {
        "slide_description": "솔루션이 실제로 어떻게 작동하고 활용될 수 있는지를 시나리오 기반으로 보여줍니다. (예시)",
        "description": "",
        "elements": {
            "Title": "슬라이드 제목 (예시)",
            "MiddleText": "시나리오 요약 (예시)",
            "ScenarioDiagram": {
                "description": "워크플로우 또는 사용자 여정 흐름도 (예시)",
                "visual_type": "사용자 Journey Map / Swimlane / Flowchart (예시)",
                "steps": ["고객 요청 (예시)", "시스템 분석 (예시)", "자동 처리 (예시)", "결과 제공 (예시)"]
            },
            "NarrativeCases": {
                "description": "시나리오 2~3개 구성 (업무 상황 → 해결 흐름 → 개선 효과) (예시)",
                "format": [
                    {"Title": "고객 VOC 자동 처리 (예시)", "Before": "수작업으로 2일 소요 (예시)", "After": "자동 분류 및 1시간 내 응답 (예시)"},
                    {"Title": "인증 시스템 통합 (예시)", "Before": "이중 로그인 발생 (예시)", "After": "SSO 기반 사용자 편의성 향상 (예시)"}
                ]
            },
            "PersonaIcons": "사용자 유형별 아이콘 및 활용 방식 요약 (예: 실무자, 관리자) (예시)",
            "ExpectedOutcomes": "각 시나리오별 기대 효과 (시간 단축, 오류 감소 등) (예시)",
            "ProcessExceptionHandling": "예외 상황 처리 플로우 (에러 발생 시 대응 흐름 등) (예시)"
        },
        "needs_research": []
    },
    "strategic_recommendations": {
        "slide_description": "고객을 위한 전략적 방향성과 구체 실행안, 우선순위를 제시합니다. (예시)",
        "description": "",
        "elements": {
            "Title": "슬라이드 제목 (예시)",
            "MiddleText": "전략 방향 요약 (예시)",
            "BulletPoints": "주요 전략 항목 (예시)",
            "PriorityMap": "우선순위 매트릭스 (예시)",
            "StrategicThemes": "전략을 묶는 키 테마 (예: 자동화, 통합, 사용자 중심) (예시)",
            "ActionRoadmap": "각 전략별 실행 계획 한 줄 요약 (예시)",
            "ClientTailoredValue": "고객 특성 반영 전략 가치 명시 (ex: '공공기관의 규제 대응에 특화') (예시)"
        },
        "needs_research": []
    },
    "implementation_plan": {
        "slide_description": "전략 실행을 위한 단계별 로드맵을 구체적으로 설명합니다. (예시)",
        "description": "",
        "elements": {
            "Title": "슬라이드 제목 (예시)",
            "MiddleText": "실행 로드맵 핵심 요약 (예시)",
            "TimelineMain": {
                "description": "전체 일정 요약 (Gantt 형식 또는 단계별 막대 도식) (예시)",
                "period": "6개월 ~ 1년 제안 가능 (예시)",
                "visual_type": "Phase Timeline / Roadmap Chart (예시)"
            },
            "Phases": {
                "description": "각 단계 명칭, 기간, 주요 작업 정리 (예시)",
                "format": [
                    {"Phase": "1단계: 기획/설계 (예시)", "Duration": "~2개월 (예시)", "Details": "요구사항 정의 및 상세 설계 (예시)"},
                    {"Phase": "2단계: 구축/테스트 (예시)", "Duration": "~3개월 (예시)", "Details": "솔루션 개발, 검증 (예시)"},
                    {"Phase": "3단계: 운영이관 (예시)", "Duration": "~1개월 (예시)", "Details": "고객 교육 및 운영 매뉴얼 전달 (예시)"}
                ]
            },
            "TextBottom": "주요 리스크 및 대응 방안, 기술/인력 관련 고려 사항 (예시)",
            "ImplementationTeam": {
                "description": "각 단계별 역할 및 인력 투입 계획 (PM, 개발, QA 등) (예시)",
                "table_format": [["단계", "역할", "투입 인원 (예시)"], ["1단계", "PM", "1 (예시)"], ["2단계", "개발자", "3 (예시)"]]
            },
            "MilestoneIcons": "각 주요 이벤트에 대한 시각 아이콘 (Kickoff, UAT 등) (예시)",
            "ClientCollaborationPoints": "고객과의 협업 시점 명시 (승인, 테스트 등) (예시)",
            "SuccessCriteria": "각 단계별 완료 기준 또는 KPI 정의 (예시)"
        },
        "needs_research": []
    },
    "timeline_milestones": {
        "slide_description": "전체 일정에서의 주요 마일스톤을 시각화하여 이해도를 높입니다. (예시)",
        "description": "",
        "elements": {
            "Title": "슬라이드 제목 (예시)",
            "MiddleText": "일정 요약 또는 주요 포인트 (예시)",
            "GanttChart": {
                "description": "기간별 주요 활동과 마일스톤 도식화 (예시)",
                "chart_type": "Gantt / Roadmap / Phase-based Timeline (예시)",
                "milestones": [
                    {"Date": "2025.07 (예시)", "Label": "Kick-off (예시)"},
                    {"Date": "2025.08 (예시)", "Label": "1차 개발 완료 (예시)"},
                    {"Date": "2025.10 (예시)", "Label": "통합 테스트 시작 (예시)"},
                    {"Date": "2025.12 (예시)", "Label": "최종 이관 (예시)"}
                ]
            },
            "MilestoneIcons": "각 마일스톤에 대한 시각 아이콘 또는 상태 표시 (착수, 승인, 완료 등) (예시)",
            "DependencyNote": "단계 간 종속 관계 또는 병렬 처리 여부 요약 (예시)",
            "ClientCheckpoints": "고객 승인/참여 포인트 명시 (UAT, 검토 회의 등) (예시)"
        },
        "needs_research": []
    },
    "risk_management_plan": {
        "slide_description": "예상되는 리스크를 식별하고 이에 대한 대응 전략을 제시합니다. (예시)",
        "description": "",
        "elements": {
            "Title": "슬라이드 제목 (예시)",
            "MiddleText": "리스크 관리 개요 (예시)",
            "RiskTable": "리스크 목록 (내용 | 가능성 | 영향도) (예시)",
            "MitigationStrategy": "완화 전략 항목 (예시)"
        },
        "needs_research": []
    },
    "expected_benefits": {
        "slide_description": "제안서 실행 시 고객이 얻게 될 주요 기대 효과를 정성·정량적으로 표현합니다. (예시)",
        "description": "",
        "elements": {
            "Title": "슬라이드 제목 (예시)",
            "MiddleText": "핵심 기대 효과 요약 (예시)",
            "BulletPoints": {
                "description": "정성적 효과 정리 (예시)",
                "examples": [
                    "운영 프로세스 간소화 및 자동화 (예시)",
                    "직원 업무 효율 증대 (예시)",
                    "고객 만족도 향상 (예시)"
                ]
            },
            "KPIProjection": {
                "description": "성과 지표 기반 기대 성과 (예시)",
                "chart_type": "Line / Bar / Table (예시)",
                "data": [["항목", "현재", "도입 후 (예시)"], ["처리 시간", "5일", "2일 (예시)"], ["운영 비용", "100%", "80% (예시)"]]
            },
            "TangibleVsIntangible": "정량 vs 정성 효과 분류 테이블 (예시)",
            "ROIIndicator": "ROI, Payback 기간 등 주요 재무적 효과 요약 (예시)",
            "ClientQuoteBox": "고객이 공감할 수 있는 기대 효과 강조 인용문 (예시)"
        },
        "needs_research": []
    },
    "investment_budget_estimation": {
        "slide_description": "전체 예산과 각 항목별 비용을 상세히 제시하여 투자 가시성을 제공합니다. (예시)",
        "description": "",
        "elements": {
            "Title": "슬라이드 제목 (예시)",
            "MiddleText": "투자 요약 (예시)",
            "BudgetTable": "총 예산 테이블 (예시)",
            "CostBreakdown": "비용 항목별 상세 내용 (예시)"
        },
        "needs_research": []
    },
    "team_introduction": {
        "slide_description": "수행팀의 전문성과 역할 분담을 보여줘 신뢰도를 높입니다. (예시)",
        "description": "",
        "elements": {
            "Title": "슬라이드 제목 (예시)",
            "MiddleText": "팀 구성 및 핵심 인력 요약 (예시)",
            "MemberList": "핵심 인력 소개 (이름, 역할, 경력 등) (예시)",
            "TeamOrgChart": "팀 조직도 (예시)"
        },
        "needs_research": []
    },
    "why_us_differentiation": {
        "slide_description": "자사만의 차별성과 강점을 강조하여 경쟁사 대비 우위를 설득합니다. (예시)",
        "description": "",
        "elements": {
            "Title": "슬라이드 제목 (예시)",
            "MiddleText": "자사 강점 요약 (예시)",
            "BulletPoints": {
                "description": "차별화된 요소 요약 리스트 (예시)",
                "examples": [
                    "업계 최고 수준의 기술 인력 및 구축 경험 (예시)",
                    "공공/금융 분야 다수의 실증 사례 (예시)",
                    "전담 PM 배정 및 신속한 커뮤니케이션 구조 (예시)",
                    "End-to-End 서비스 제공 (컨설팅 ~ 운영까지) (예시)"
                ]
            },
            "ComparisonTable": {
                "description": "자사 vs 경쟁사 비교 (예시)",
                "table_format": [
                    ["항목", "자사 (예시)", "경쟁사 (예시)"],
                    ["경험", "10건 이상 유사 구축 (예시)", "2~3건 제한적 (예시)"],
                    ["대응 속도", "24시간 이내 피드백 (예시)", "72시간 이상 (예시)"]
                ]
            }
        },
        "needs_research": []
    },
    "closing_summary": {
        "slide_description": "전체 제안을 요약하며 고객에게 남기고 싶은 핵심 메시지를 강조합니다. (예시)",
        "description": "",
        "elements": {
            "Title": "슬라이드 제목 (예시)",
            "MiddleText": "핵심 요약 문장 (예시)",
            "FinalCall": {
                "description": "마무리 제안 또는 CTA (예시)",
                "options": [
                    "시범 도입 제안 (예시)",
                    "추가 미팅 요청 (예시)",
                    "RFP 연장 논의 (예시)",
                    "기술 검증(PoC) 요청 등 (예시)"
                ]
            },
            "ThankYouVisual": "감사 인사와 함께 신뢰를 전달하는 이미지 또는 문구 (예: 함께 성장할 파트너) (예시)",
            "ContactInfoBox": "담당자 연락처 정보 (이름, 이메일, 직책, 회사 로고 포함) (예시)",
            "ReinforcementBanner": "슬라이드 하단에 ‘선택의 이유’를 다시 강조하는 요약 배너 (아이콘 또는 문장) (예시)"
        },
        "needs_research": []
    },
    "qna": {
        "slide_description": "질의응답을 위한 충분한 공간과 예상 질문 대응 내용을 구성합니다. (예시)",
        "description": "",
        "elements": {
            "Title": "슬라이드 제목 (예시)",
            "EmptySpace": "Q&A 시각적 공간 (예시)",
            "AnticipatedQuestions": "예상 질문 및 답변 리스트 (예시)"
        },
        "needs_research": []
    },
    "appendix": {
        "slide_description": "통계, 기술 스펙, 참고 문헌 등 본문에 넣기 어려운 자료를 정리합니다. (예시)",
        "description": "",
        "elements": {
            "Title": "슬라이드 제목 (예시)",
            "SupportingDetails": "기술 자료, 통계 인용 등 (예시)",
            "Footnotes": "출처, 링크, 각주 등 보충 정보 (예시)"
        },
        "needs_research": []
    }
}


Functions & Test

In [26]:
def extract_text_from_file(file_path):
    """
    PDF 또는 Word(.docx) 파일에서 텍스트를 추출하고, 줄바꿈 및 띄어쓰기를 정제하는 함수
    """
    from docx import Document
    from PyPDF2 import PdfReader
    ext = os.path.splitext(file_path)[-1].lower()
    text = ""

    if ext == ".pdf":
        try:
            reader = PdfReader(file_path)
            for page in reader.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text + "\n"
        except Exception as e:
            print(f"[PDF 읽기 오류] {e}")

    elif ext == ".docx":
        try:
            doc = Document(file_path)
            for para in doc.paragraphs:
                text += para.text.strip() + "\n"
        except Exception as e:
            print(f"[DOCX 읽기 오류] {e}")

    else:
        raise ValueError("지원하지 않는 파일 형식입니다. PDF 또는 DOCX만 가능합니다.")

    # 후처리: 줄바꿈 정제
    text = re.sub(r"(?<!\n)\n(?!\n)", " ", text)      # 문장 중간 줄바꿈 제거
    text = re.sub(r"\n{2,}", "\n", text)              # 문단 구분은 하나의 \n으로
    text = re.sub(r"[ \t]{2,}", " ", text)            # 연속 공백 제거

    return text.strip()

In [27]:
# test extract_text_from_file
pdf_path = "DB/RFP/25년 삼성전자 MX 미국 직영 매장 PMO_입찰공고문_F.docx"
rfp_text = extract_text_from_file(pdf_path)
rfp_text[:100]

'프로젝트 배경 및 운영 목표 - 삼성전자 직영 매장의 성공적인 구축 및 운영 모델 확보가 필요 . 제조업과 달리 직영 리테일은 당사에게는 신규 사업 분야임 . 내/외부 다양한 전문'

In [28]:
def get_user_input(
    rfp_text=None,
    style_selected=None,
    keywords_input=None,
    client_name=None,
    proposal_title=None,
    user_direction=None
):
    """
    사용자 입력 기반 제안서 생성용 입력값 정리 함수
    추후 고객 제안서 데이터 추가등을 통해 고도화 예정
    
    Args:
        rfp_text (str): RFP 원문 텍스트
        style_selected (str): 제안서 스타일 ("격식 있는", "신뢰감 있는" 등)
        keywords_input (str): 강조 키워드 쉼표 구분 (예: "AI, LLM, 효율성")
        client_name (str): 고객사명
        proposal_title (str): 제안서 제목
        user_direction (str): 고객 요청 방향성 
        ***(제안 방안 장성시 해당 방안으로 적용되는지 확인 필요)

    Returns:
        dict: 제안서 생성용 파라미터
    """

    if not rfp_text:
        raise ValueError("⚠️ RFP 텍스트는 필수입니다.")

    style_selected = style_selected or "신뢰감 있는"
    keywords_list = [kw.strip() for kw in (keywords_input or "").split(",") if kw.strip()]
    client_name = client_name or "고객사명 미입력"
    proposal_title = proposal_title or "제안서 제목 미입력"
    user_direction = user_direction or ""

    return {
        "rfp_text": rfp_text,
        "style": style_selected,
        "keywords": keywords_list,
        "client_name": client_name,
        "proposal_title": proposal_title,
        "user_direction": user_direction
    }


In [29]:
# test RFP 텍스트와 일부 값 입력
sample_rfp_text = rfp_text

user_inputs = get_user_input(
    rfp_text=sample_rfp_text,
    style_selected="근거가 있고 formal하게",
    keywords_input="타사 비교, 자체적인, 벤치마크",
    client_name="EY 컨설팅",
    proposal_title="삼성전자 MX 미국 직영 매장 PMO 프로젝트",
    user_direction="현실적으로 운영 가능하고 모두가 납득할만한 근거를 가진 맞춘 설계 필요"
)

# ✅ 출력 확인
for k, v in user_inputs.items():
    print(f"\n🔹 {k}:\n{v if not isinstance(v, str) else v[:50]}")


🔹 rfp_text:
프로젝트 배경 및 운영 목표 - 삼성전자 직영 매장의 성공적인 구축 및 운영 모델 확보가 

🔹 style:
근거가 있고 formal하게

🔹 keywords:
['타사 비교', '자체적인', '벤치마크']

🔹 client_name:
EY 컨설팅

🔹 proposal_title:
삼성전자 MX 미국 직영 매장 PMO 프로젝트

🔹 user_direction:
현실적으로 운영 가능하고 모두가 납득할만한 근거를 가진 맞춘 설계 필요


In [30]:
def analyze_rfp(
    user_inputs: dict,
    slide_templates: dict,
    model="gpt-4o",
    temperature=0.3
):
    """
    get_user_input 결과 dict를 받아 GPT로 분석하여 PPT 슬라이드 초안 생성

    Args:
        user_inputs (dict): get_user_input 함수에서 반환된 사용자 입력값
        slide_templates (dict): PPT 템플릿 구조
        model (str): 사용할 OpenAI 모델
        temperature (float): 생성 자유도

    Returns:
        dict: 슬라이드별 제안서 구성 내용
    """
    rfp_text = user_inputs.get("rfp_text")
    style = user_inputs.get("style", "신뢰감 있는")
    keywords = user_inputs.get("keywords", [])
    client_name = user_inputs.get("client_name", "고객사명 미입력")
    proposal_title = user_inputs.get("proposal_title", "제안서 제목 미입력")
    user_direction = user_inputs.get("user_direction", "")

    if not rfp_text or len(rfp_text.strip()) < 30:
        raise ValueError("❗ RFP 원문이 비어 있거나 너무 짧습니다. 실제 RFP를 반드시 입력하세요.")
    if slide_templates is None:
        raise ValueError("PPT 템플릿 구조 딕셔너리가 필요합니다.")

    # system prompt -> 추후 고객 ppt 데이터 활용
    system_prompt = f"""
    당신은 'EY·맥킨지 등 국내외 최상위 전략 컨설팅 회사의 파트너급 제안서 전문가 AI'입니다.
    
    아래 RFP 원문과 고객 요구를 바탕으로,
    RFP 본문을 보고 각 PPT 슬라이드별로 실제 컨설팅 현장과 동일한 수준의, 논리적이고 설득력 있는 제안서 초안 구조를 작성하세요.
    
    - 슬라이드 순서와 템플릿 활용은 RFP의 논리적 흐름, 고객의 의사결정 포인트, 설득 전략에 따라 자유롭게 조정하세요.
    - 하나의 템플릿을 여러 번 사용하거나, 필요에 따라 생략·병합해도 무방합니다.
    - 각 슬라이드의 elements에는 실제 발표자료처럼 활용 가능한 표, 그래프, 수치, 근거, 사례, 시각적 다이어그램, 실제 데이터, 실무 워딩을 최대한 풍부하게 포함하세요. 단, 내용이 적절하지 않으면 변경해도 무방합니다.
    - elements에 추가가 필요한 사항이 있으면 추가해주세요. 필요 없는 elements는 과감히 제거해주세요.
    - 모든 슬라이드는 고객사 임원·실무자 모두를 설득할 수 있도록 전략적 논리, 정량/정성 근거, 차별화 요인, 업계 사례, 실무 설득력 기반으로 작성되어야 하며, 평가자와 실무자가 모두 납득할 수 있어야 합니다.
    
    1. [description]
    - 단순 목적 설명이 아닌: '왜 중요한가', '고객사 상황과의 연계성', '전략적 필요성', '차별화된 근거', '시장 및 경쟁사 변화와 연결성'을 반드시 포함하세요.
    - 시장 통계, 업계 사례, 정책·기술 트렌드, 고객사 현황 등 객관적 근거를 적극 인용하세요.
    - 3~5문장 이상의 깊이 있는 발표용 컨설팅 워딩으로 작성하세요.
    - 슬라이드의 논리적 흐름과 고객 의사결정 포인트와의 연결을 명확히 하세요.
    - cover_page, table_of_contents를 제외한 항목에는 모두 description이 상세히 설명되어야합니다.
    
    2. [elements]
    - 실제 발표자료처럼 사용 가능한 수준으로 표, 수치, 근거, 사례, 시각적 다이어그램, 실제 데이터, 실무 워딩을 최대한 풍부하게 포함하세요.
    - 표, 그래프, 다이어그램 등 시각적 요소는 반드시 포함하고, 실제 데이터와 출처를 명확히 기재하세요.
    - 표는 다음 형식으로 작성:
      "표 제목": [["헤더1", "헤더2"], ["값1", "값2"]]
    - 그래프는 다음 형식으로 작성: (예시)
      "그래프 제목": {{
        "description": "시장 성장률 추이 (2019~2024)",
        "graph_type": "Line Chart",
        "data_source": "Statista, 2023",
        "data_table": [["연도", "시장 규모 (억원)"], ["2019", "1200"], ["2020", "1400"]]
      }}
    - 실제 업계 사례, 벤치마크, 성공/실패 요인, 고객사 맞춤형 인사이트 등도 반드시 포함하세요.
    - description의 핵심 논리와 근거가 elements에도 반드시 반영되어야 하며, 슬라이드의 설득 포인트가 명확히 드러나야 합니다.
    - 최종 제안서는 elements 내 데이터를 사용하니 최대한 구체적이고 상세히 작성해주세요.
    
    3. [Title]
    - 각 슬라이드 description과 slide_id에 적합한 내용으로, 한 번에 해당 슬라이드의 핵심 메시지와 의미가 명확히 드러나도록 작성하세요.
    - slide_id와 동일한 표현은 사용하지 마세요. (예: Project Understanding 금지)
    - 고객사 임원·실무자가 슬라이드 제목만 보고도 내용을 직관적으로 이해할 수 있어야 합니다.
    
    4. [MiddleText]
    - 대부분의 ppt에 들어가는 elements로 Title 아래 최대한 상세히 Title과 슬라이드를 설명하는 역할을 합니다.
    
    5. [requirement_detail] 템플릿 규칙
    - client_needs_summary에 있는 NeedsBulletPoints 항목을 각각 슬라이드로 작성해야 합니다.
    - 어떻게 고객의 요구사항을 충족할지 상세히 작성합니다.
    - 즉, 각 bullet에 대해 별도의 requirement_detail 슬라이드를 만들어야 하며, 다음과 같은 구조를 따라야 합니다:
    
    예시:
    "requirement_detail_01": {{
      "slide_description": "...",
      "description": "...",
      "elements": {{
        "Title": "...",
        "MiddleText": "...",
        "ResponsePlanBox": "..."
        ...
      }},
      "needs_research": ["조사 필요 항목"]
    }}
    "requirement_detail_02": {{
      "slide_description": "...",
      "description": "...",
      "elements": {{
        "Title": "...",
        "MiddleText": "...",
        "ResponsePlanBox": "..."
        ...
      }},
      "needs_research": ["조사 필요 항목"]
    }}
    추가 지침:
    - 슬라이드별로 논리적 연결성(Why → What → How → So What → Next)을 고려해 작성하세요.
    - 고객사 맞춤형 메시지, 업계 트렌드, 경쟁사 동향, 차별화 전략, 실제 수치와 사례, 시각적 설득력을 모두 반영하세요.
    - 모든 슬라이드는 실제 컨설팅사 파트너가 직접 리뷰·수정하는 수준의 품질을 목표로 하세요.
    - 모든 내용은 반드시 실제 데이터, 실제 사례, 실무적 설득력을 기반으로 해야 하며 허위 정보는 절대 포함하지 마세요.
    - 템플릿 순서는 반드시 따를 필요 없습니다. 자유롭게 재구성하고, 필요시 반복 사용하세요.
    - 반드시 JSON 형식으로 응답해야 하며, 다른 텍스트(마크다운, 예시 설명 등)는 절대 포함하지 마세요.
    
    [슬라이드 템플릿 구조]
    {json.dumps(slide_templates, ensure_ascii=False, indent=2)}
    """.strip()

    # user prompt
    user_prompt = f"""
    [RFP 원문]
    {rfp_text}
    
    [고객 방향성/강조]
    {user_direction or '없음'}
    
    (고객명: {client_name} / 제안서 제목: {proposal_title})
    [강조 키워드]: {', '.join(keywords) if keywords else '없음'}
    
    위 정보를 바탕으로 PPT 슬라이드별 'description', 'elements', 'needs_research'를 작성해주세요.
    """.strip()

    # GPT 호출
    response = openai.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=temperature
    )

    result_text = response.choices[0].message.content.strip()

    # 코드블럭 제거
    result_text = re.sub(r"^```(?:json)?\s*|\s*```$", "", result_text, flags=re.DOTALL)

    try:
        parsed = json.loads(result_text)
        return parsed
    except json.JSONDecodeError as e:
        raise ValueError(f"응답이 유효한 JSON 형식이 아닙니다:\n\n{result_text}\n\n에러: {e}")


In [31]:
# analyze_rfp test
slides_dict = analyze_rfp(user_inputs=user_inputs, slide_templates=PROPOSAL_SLIDE_TEMPLATES)
slides_dict

{'cover_page': {'elements': {'Title': '삼성전자 MX 미국 직영 매장 PMO 프로젝트',
   'Subtitle': 'EY 컨설팅',
   'ProjectDate': '2025년 6월 25일',
   'PreparedBy': 'EY 컨설팅',
   'Logo': '삼성전자 및 EY 로고'}},
 'table_of_contents': {'slide_description': '전체 제안서의 슬라이드 구성을 한눈에 파악할 수 있도록 시각적으로 정리합니다.',
  'description': '',
  'elements': {'Title': '목차',
   'SectionList': ['Executive Summary',
    'Project Understanding',
    'Client Needs Summary',
    'Requirement Details',
    'Market Analysis & Trends',
    'Strategic Recommendations',
    'Implementation Plan',
    'Risk Management Plan',
    'Expected Benefits',
    'Investment & Budget Estimation',
    'Team Introduction',
    'Why Us & Differentiation',
    'Closing Summary',
    'Q&A',
    'Appendix']}},
 'executive_summary': {'slide_description': '제안서 전체의 핵심 내용을 1~2페이지 내에 요약하여 임원 또는 의사결정자가 빠르게 이해할 수 있도록 구성합니다.',
  'description': '삼성전자의 미국 직영 매장 구축 및 운영을 위한 프로젝트는 제조업과는 다른 리테일 분야의 새로운 도전입니다. EY 컨설팅은 선진사 벤치마크와 트렌드 분석을 통해 성공적인 매장 운영 모델을 제안합니다. 본 프로젝트는 4개월 간의 계약 

In [35]:
# 슬라이드 딕셔너리 구조를 전개하여 테이블 형태로 변환
def flatten_slides_dict(slides_dict):
    slide_rows = []
    for sid, slide in slides_dict.items():
        if isinstance(slide, dict):
            slide_rows.append({
                "slide_id": sid,
                "description": slide.get("description", ""),
                "elements": slide.get("elements", {})
            })
    return pd.DataFrame(slide_rows)


def generate_research_questions(slide_title, description, elements_dict):
    element_summary = "\n".join([f"{k}: {v}" for k, v in elements_dict.items()])
    prompt = (
        "당신은 전문 컨설팅 회사의 제안서를 작성하는 리서치 전문가입니다.\n"
        "template 내 cover_page, table_of_contents는 질문 생성하지 마세요.\n"
        "elements내 수치 관련 내용을 확인 할 수 있도록 질문을 생성해주세요.\n"
        "필요한 경우가 아니면 고객사의 이름이 들어간 질문은 생성하지 말고 범용적 질문을 만드세요. ex). EY 컨설팅\n"
        "아래는 특정 슬라이드의 설명과 구성 요소입니다. 이 내용을 기반으로 외부 리서치가 필요한 항목을 식별하고, 그에 대한 질문을 작성해 주세요.\n\n"
        "[지침]\n"
        "1. 수치, 시장 규모, 성장률, 산업 트렌드, 경쟁사 비교 등 외부 데이터를 통해 검증 가능한 항목에 집중하세요.\n"
        "2. 단순 설명이나 내부 고유 내용은 제외하고, 외부 참고 자료가 필요한 항목만 질문하세요. \n"
        "3. 그래프나 표가 포함된 경우, 'data_source', 'data_table'의 신뢰성 및 수치 검증을 위한 질문을 포함하세요.\n"
        "4. 경쟁사 비교와 같이 구체적 경쟁사가 필요 할 때는 '경쟁사 = 이 산업의 대표 기업 (예: 애플, 구글 등)'처럼 구체적인 대상 및 기업의 정보가 포함되어야 합니다.\n"
        "5. 질문 수는 슬라이드당 1~3개 이내로 제한하며, 핵심만 담은 간결한 문장으로 작성해주세요.\n"
        "6. 질문이 필요 없는 슬라이드일 경우, 질문 없이 넘어가 주세요.\n"
        "7. 시장규모, 성장률 등 수치 데이터는 꼭 확인하기 위한 질문이 필요합니다. ex). market_analysis_market_overview -> 스마트폰 2020년 시장규모 2000억 -> 2조3000억\n"
        "8. 경쟁사, 벤치 마킹시 명확한 경쟁사를 찾아 질문해주세요. 경쟁사 A -> 애플\n"
        "9. research 내용이 반드시 슬라이드의 description과 elements 내용과 관련 있는 데이터를 사용해주세요. ex).'AI 기술 시장 성장률 추이는? -> 'AI 학습 데이터 시장 성장률 추이 결과'인 경우 '아니오'\n"
        "10. requirement_detail 내용이 고객에게 설득이 되도록 NeedsBulletPoints의 내용을 구체화하여 질문을 만들어야한다.\n"  

        "11. 질문은 영어로 작성해주세요.\n\n"
        f"[슬라이드 제목]: {slide_title}\n"
        f"[슬라이드 설명]: {description}\n"
        f"[슬라이드 요소]:\n{element_summary}\n\n"
        "이 슬라이드의 내용을 외부 정보로 검증하기 위해 필요한 리서치 질문은 무엇입니까?"
    )
    response = openai.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3
    )
    result = response.choices[0].message.content.strip()
    return [line.strip("-•1234567890. ").strip() for line in result.split("\n") if "?" in line]

def generate_needs_research_table_from_df(slides_df):
    needs_list = []
    for _, row in slides_df.iterrows():
        title = row.get("elements", {}).get("Title", "")
        desc = row.get("description", "")
        elements = row.get("elements", {})
        questions = generate_research_questions(title, desc, elements)
        needs_list.append(questions)
    slides_df["needs_research"] = needs_list
    return slides_df

def display_slides_with_research(slides_df):
    for idx, row in slides_df.iterrows():
        print(f"\n📌 Slide ID: {row['slide_id']}")
        print(f"설명: {row.get('description', '[없음]')}")
        print("🔍 리서치 질문:")
        if isinstance(row.get("needs_research", []), list) and row["needs_research"]:
            for q in row["needs_research"]:
                print(f" - {q}")
        else:
            print(" - (없음)")

def inject_research_questions_to_dict(slides_dict, slides_df):
    """
    slides_df의 needs_research 값을 slide_id 기준으로 slides_dict에 다시 삽입
    """
    for _, row in slides_df.iterrows():
        sid = row["slide_id"]
        questions = row.get("needs_research", [])
        if isinstance(questions, str):
            # 문자열로 되어 있는 경우 리스트로 변환 (ex. "['질문1', '질문2']")
            import ast
            try:
                questions = ast.literal_eval(questions)
            except Exception:
                questions = [questions]
        if isinstance(questions, list):
            slides_dict[sid]["needs_research"] = questions
    return slides_dict

In [36]:
slides_df = flatten_slides_dict(slides_dict)

In [37]:
slides_df = generate_needs_research_table_from_df(slides_df)

In [39]:
display_slides_with_research(slides_df)


📌 Slide ID: cover_page
설명: 
🔍 리서치 질문:
 - (없음)

📌 Slide ID: table_of_contents
설명: 
🔍 리서치 질문:
 - (없음)

📌 Slide ID: executive_summary
설명: 삼성전자의 미국 직영 매장 구축 및 운영을 위한 프로젝트는 제조업과는 다른 리테일 분야의 새로운 도전입니다. EY 컨설팅은 선진사 벤치마크와 트렌드 분석을 통해 성공적인 매장 운영 모델을 제안합니다. 본 프로젝트는 4개월 간의 계약 기간 동안 매장 운영 전 영역의 매뉴얼화 및 프로세스 확립을 목표로 하며, 글로벌 표준 가이드라인을 수립합니다.
🔍 리서치 질문:
 - What are the current trends and best practices in retail store operations in the United States that can be used as benchmarks for Samsung's direct store operations?
 - How do leading companies in the retail industry, such as Apple and Amazon, structure their store operation manuals and processes to ensure efficiency and effectiveness?
 - What are the global standard guidelines for retail store operations that can be applied to enhance Samsung's competitiveness in the global market?

📌 Slide ID: project_understanding
설명: 삼성전자는 제조업에서 리테일로의 확장을 통해 새로운 사업 기회를 모색하고 있습니다. 직영 매장 운영은 고객 경험을 직접 관리하고 브랜드 가치를 높이는 중요한 전략적 요소입니다. 본 프로젝트는 미국 내 신규 매장 오픈과 기존 파트너 매장의

In [40]:
# 생성한 research 질문 -> slides_dict 포멧 needs_research에 추가
slides_dict_updated = inject_research_questions_to_dict(slides_dict, slides_df)

Research 자동화

In [41]:
def search_perplexity(query, api_key):
    url = "https://api.perplexity.ai/chat/completions"
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": "sonar-pro",
        "messages": [{"role": "user", "content": query}],
        "return_citations": True  # 중요: citation 포함하도록 설정
    }

    response = requests.post(url, headers=headers, json=payload)
    if response.status_code == 200:
        try:
            data = response.json()
            message = data.get("choices", [{}])[0].get("message", {})
            content = message.get("content", "")

            # ✅ 실제 citation URL은 'search_results' 필드에 있음
            search_results = data.get("search_results", [])
            urls = []
            for result in search_results:
                url = result.get("url")
                if url:
                    urls.append(url)

            return content.strip(), urls
        except Exception as e:
            print("Perplexity Parsing Error:", e)
            return "", []
    else:
        print("Perplexity API Error:", response.status_code, response.text)
        return "", []


def search_serpapi_with_url(query, serpapi_key):
    """
    SerpAPI를 사용하여 Google 검색 결과 요약과 함께 URL 반환
    """
    url = "https://serpapi.com/search"
    params = {
        "engine": "google",
        "q": query,
        "api_key": serpapi_key
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        results = response.json()
        snippets = []
        urls = []
        for r in results.get("organic_results", [])[:3]:
            snippet = r.get("snippet", "")
            link = r.get("link", "")
            if snippet and link:
                snippets.append(snippet)
                urls.append(link)
        return "\n".join(snippets), urls
    else:
        print("SerpAPI Error:", response.status_code, response.text)
        return "", []

def extract_relevant_summary_from_content(slide_description, elements, research_result):
    """
    리서치 결과 중 slide_description 및 elements 내용에 부합하는 핵심 정보만 요약 추출
    단, 요약은 지나치게 축약되지 않도록 3~5문장 내외로 간결히 작성
    """
    prompt = (
        f"당신은 제안서 작성 지원 AI입니다.\n"
        f"다음은 슬라이드의 목적과 포함된 요소들, 그리고 리서치 결과입니다.\n\n"
        f"[슬라이드 목적]\n{slide_description}\n\n"
        f"[슬라이드 요소]\n{elements}\n\n"
        f"[리서치 결과]\n{research_result}\n\n"
        f"리서치 결과중 출처가 blog와 같이 신뢰하기 힘든 데이터는 제외해주세요.\n\n"
        f"리서치 결과와 elements내 들어가는 내용, url을 꼭 매칭해주세요. ex). [ROI 20%, AI 기반 자동화 모듈을 통해 업무 효율성을 크게 높인 바 있으며, 이는 ROI 20%와 TCO 절감 15%를 달성하는 데 기여합니다., https://botpress.com/ko/blog/top-artificial-intelligence-trends\n\n"
        f"리서치 결과중 장표에 들어가야하는 수치는 모두 research_result에서 나온 수치가 꼭 포함이 될수 있도록 해주세요 ex). 성장률, 효율성 향상률, 시장 규모 등\n\n"
        f"위 슬라이드 목적과 elements와 슬라이드 목적에 부합하는 핵심 정보만 요약하세요. 단, elements내 수치 데이터 및 필요한 데이터는 꼭 포함하세요.\nn"
        f"불필요한 일반론, 도입부, 중복 표현은 제거하세요. 요약은 3~5문장으로 유지하세요."
    )
    response = openai.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3
    )
    return response.choices[0].message.content.strip()


def check_relevance(description, elements, research_result):
    """
    리서치 결과가 slide_description과 논리적으로 부합하는지 확인
    """
    prompt = (
        "당신은 전략 컨설팅 제안서 검토 전문가입니다.\n\n"
        "다음은 특정 슬라이드의 목적 설명과, 최신 리서치 결과입니다. 슬라이드의 elements는 기존 GPT 모델 기반 초안이며, 정보가 오래되었거나 덜 유의미할 수 있습니다.\n"
        "반면 아래 리서치 결과는 최신이고, 신뢰 가능한 정보원에서 가져왔습니다.\n\n"
        "[슬라이드 목적]\n"
        f"{description}\n\n"
        "[슬라이드 내용]\n"
        f"{elements}\n\n"
        "[리서치 결과]\n"
        f"{research_result}\n\n"
        "📌 검토 기준:\n"
        "- 리서치 결과가 슬라이드의 목적에 실질적으로 기여하거나, 슬라이드에 포함되면 정보의 품질이나 설득력이 향상되는 경우, '네'라고 답하십시오.\n"
        "- 단순히 배경 설명 수준이거나 목적과 동떨어진 내용이면 '아니오'로 답하십시오.\n"
        "- 질문과 관련 없는 데이터의 경우 과감하게 제외해주세요. ex). AI 모델 학습 및 평가 자동화의 현재 시장 규모와 관련 없는 결과  '질문: AI 모델 학습 및 평가 자동화의 현재 시장 규모와 향후 5년간의 예상 성장률은 어떻게 되나요?' 답변: 'AI 교육 데이터 세트 시장은 2024년 292억 달러에서 2032년 1,740억 달러로 성장할 전망이며, 서비스형 AI 시장은 2024년 127억 달러에서 연평균 30.6% 성장할 것으로 예상됩니다.'\n"
        "- 수치 차이나 정확도보다 슬라이드 작성에 도움이 되는 실용적 정보인지를 중점적으로 판단하세요.\n\n"
        "답변은 반드시 '네' 또는 '아니오'로 시작하고, 1~2문장으로 이유를 설명해주세요.\n"
        "예시: 네. 이 데이터는 시장 성장률과 관련된 정보로, 해당 슬라이드의 설득력을 높이는 데 유용합니다."
    )

    response = openai.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.1
    )
    answer = response.choices[0].message.content.strip()
    print(f"\n[GPT 판단 결과]\n{answer}\n")
    return answer.startswith("네")


def regenerate_query(original_query, previous_answer):
    """
    검색 결과가 부적합한 경우, 질문을 더 명확하게 만들어주는 로직
    """
    prompt = (
        f"다음은 정보 검색을 위한 원래 질문과 검색 결과입니다.\n\n"
        f"[질문]\n{original_query}\n\n"
        f"[검색 결과]\n{previous_answer}\n\n"
        "위 결과가 부정확하거나 부족한 경우, 질문을 더 구체적이고 답변을 유도할 수 있도록 재작성하세요. "
        "새 질문은 1문장으로 간결하게 작성해 주세요."
    )
    response = openai.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3
    )
    return response.choices[0].message.content.strip()

def smart_research_fill(slides_dict, search_mode="perplexity", max_retry=2):
    """
    슬라이드에 대해 스마트 리서치를 수행하고, 관련성 판단 결과를 슬라이드 제목과 질문과 함께 출력함.
    Perplexity 사용 시 citation URL도 함께 저장됨.
    """
    completed_results = {}

    for slide_id, slide in slides_dict.items():
        desc = slide.get("description", "")
        if not desc or not isinstance(desc, str):
            print(f"[경고] description이 비어있거나 문자열이 아님: slide_id={slide_id}")
            continue

        needs_list = slide.get("needs_research", [])
        if not isinstance(needs_list, list) or not needs_list:
            print(f"[스킵] needs_research 없음 또는 빈 리스트 - slide_id={slide_id}")
            continue

        slide_title = slide.get("slide_title", slide_id)
        slide_elements = slide.get("elements", {})

        # JSON 안전하게 변환
        try:
            elements_str = json.dumps(slide_elements, ensure_ascii=False, indent=2)
        except Exception as e:
            print(f"[고도화 실패] {slide_id}: elements 변환 실패 - {e}")
            continue

        slide_results = {}
        for need in needs_list:
            if not isinstance(need, str):
                continue
            query = need
            attempt = 0

            while attempt < max_retry:
                # 🔍 검색 수행
                if search_mode == "perplexity":
                    content, urls = search_perplexity(query, perplexity_api_key)
                elif search_mode == "serpapi":
                    content, urls = search_serpapi_with_url(query, serp_api_key)
                else:
                    raise ValueError("search_mode는 'perplexity' 또는 'serpapi'만 지원됩니다.")

                # ⛔ 검색 실패 처리
                if not content:
                    print(f"[실패] 검색 결과 없음 - query: {query}")
                    break

                # ✅ 판단 로그 출력
                print(f"\n===== [슬라이드: {slide_title}] 질문: \"{query}\" 에 대한 판단 =====")
                is_relevant = check_relevance(desc, slide_elements, content)

                if is_relevant:
                    try:
                        summary = extract_relevant_summary_from_content(
                            slide_description=desc,
                            elements=elements_str,  # 문자열로 전달
                            research_result=content
                        )
                    except Exception as e:
                        print(f"[요약 실패] {slide_id} - {query}: {e}")
                        break

                    slide_results[need] = {
                        "content": summary,
                        "urls": urls
                    }
                    break
                else:
                    query = regenerate_query(query, content)
                    attempt += 1
                    time.sleep(1)

        if slide_results:
            completed_results[slide_id] = {
                "slide_title": slide_title,
                "research_results": slide_results
            }

    return completed_results


def merge_research_results_into_slides(slides_dict, research_results_dict):
    """
    slides_dict 구조에 research_results_dict 데이터를 병합
    slides_dict[slide_id]['research_results'] 로 삽입
    """
    for slide_id, data in research_results_dict.items():
        if slide_id in slides_dict:
            slides_dict[slide_id]['research_results'] = data.get('research_results', {})
    return slides_dict


In [42]:
# perplexity 리서치 후 리서치 결과 slides_dict에 추가
research_results_dict = smart_research_fill(slides_dict_updated, search_mode="perplexity")

slides_dict_updated = merge_research_results_into_slides(
    slides_dict=slides_dict_updated,
    research_results_dict=research_results_dict
)

slides_dict_updated

[경고] description이 비어있거나 문자열이 아님: slide_id=cover_page
[경고] description이 비어있거나 문자열이 아님: slide_id=table_of_contents

===== [슬라이드: executive_summary] 질문: "What are the current trends and best practices in retail store operations in the United States that can be used as benchmarks for Samsung's direct store operations?" 에 대한 판단 =====

[GPT 판단 결과]
네. 리서치 결과는 최신 리테일 트렌드와 베스트 프랙티스를 제공하여 삼성전자의 미국 직영 매장 운영 모델을 더욱 혁신적이고 고객 중심적으로 설계하는 데 기여할 수 있습니다.


===== [슬라이드: executive_summary] 질문: "How do leading companies in the retail industry, such as Apple and Amazon, structure their store operation manuals and processes to ensure efficiency and effectiveness?" 에 대한 판단 =====

[GPT 판단 결과]
네. 리서치 결과는 선진사인 Apple과 Amazon의 매장 운영 매뉴얼 및 프로세스 구조에 대한 최신 정보를 제공하여, 삼성전자의 미국 직영 매장 운영 모델을 설계하는 데 실질적으로 기여할 수 있습니다.


===== [슬라이드: executive_summary] 질문: "What are the global standard guidelines for retail store operations that can be applied to enhance Samsung's competitiveness in the global market?" 에 대한 판단 =====

[GPT 판

{'cover_page': {'elements': {'Title': '삼성전자 MX 미국 직영 매장 PMO 프로젝트',
   'Subtitle': 'EY 컨설팅',
   'ProjectDate': '2025년 6월 25일',
   'PreparedBy': 'EY 컨설팅',
   'Logo': '삼성전자 및 EY 로고'},
  'needs_research': []},
 'table_of_contents': {'slide_description': '전체 제안서의 슬라이드 구성을 한눈에 파악할 수 있도록 시각적으로 정리합니다.',
  'description': '',
  'elements': {'Title': '목차',
   'SectionList': ['Executive Summary',
    'Project Understanding',
    'Client Needs Summary',
    'Requirement Details',
    'Market Analysis & Trends',
    'Strategic Recommendations',
    'Implementation Plan',
    'Risk Management Plan',
    'Expected Benefits',
    'Investment & Budget Estimation',
    'Team Introduction',
    'Why Us & Differentiation',
    'Closing Summary',
    'Q&A',
    'Appendix']},
  'needs_research': []},
 'executive_summary': {'slide_description': '제안서 전체의 핵심 내용을 1~2페이지 내에 요약하여 임원 또는 의사결정자가 빠르게 이해할 수 있도록 구성합니다.',
  'description': '삼성전자의 미국 직영 매장 구축 및 운영을 위한 프로젝트는 제조업과는 다른 리테일 분야의 새로운 도전입니다. EY 컨설팅은 선진사 벤치마크와 트렌드

In [44]:
def extract_dict_from_gpt_response(text):
    """
    GPT 응답에서 파이썬 딕셔너리 부분만 추출
    """
    # 코드블록 제거
    text = text.strip()
    if text.startswith("```"):
        text = re.sub(r"^```(python)?", "", text)
        text = re.sub(r"```$", "", text)

    # 중괄호 기반 딕셔너리 추출
    match = re.search(r"(\{.*\})", text, re.DOTALL)
    if match:
        return match.group(1)
    return text

# Research results를 통해 기존 elements 및 derscription 고도화 
def refine_slides_dict(slides_dict, rfp_text, user_inputs):
    refined_slides = deepcopy(slides_dict)

    for slide_id, slide in slides_dict.items():
        try:
            # 기본 요소 준비
            slide_title = slide.get("slide_title", "")
            description = slide.get("description", "")
            elements = slide.get("elements", {})
            research_results = slide.get("research_results", {})
            slide_description = slide.get("slide_description", {})
            safe_elements = json.dumps(elements, ensure_ascii=False)
            safe_research_results = json.dumps(research_results, ensure_ascii=False)

            prompt = f"""
            당신은 맥킨지 수준의 컨설팅 제안서 전략 전문가입니다.
            
            당신의 역할은 제안서 슬라이드의 내용을 전략적으로 더 설득력 있고 고도화된 형태로 다듬는 것입니다.
            research_results를 통해 description과 elements 내용 개선이 필요합니다.
            
            --- 컨텍스트 ---
            클라이언트: {user_inputs['client_name']}
            제안서 제목: {user_inputs['proposal_title']}
            문체 및 톤: {user_inputs['style']}
            작성 방향: {user_inputs['user_direction']}
            강조할 키워드: {", ".join(user_inputs['keywords'])}
            RFP 원문: {rfp_text}
            
            --- 슬라이드 정보 ---
            슬라이드 ID: {slide_id}
            슬라이드 제목: {slide_title}
            현재 슬라이드 설명: {description}
            슬라이드 설명: {slide_description}
            슬라이드 요소: {safe_elements}
            리서치 결과: {safe_research_results}
            
            --- 작업 지시 ---
            - "slide_description": 슬라이드 목적에 맞게 전략적으로 다듬은 설명(3~5문장)
            - 기존 ppt description은 삭제 
            - cover_page에는 기존 elements 존재해야함. ex). title, subtitle, projectdate, etc.
            - "elements": research_results를 기반으로 고도화된 구체적인 내용 (예: 수치 포함 표, 그래프, 비교 등)
            - resuearch_results내 slide_description과 관련 없는 경우, 다른 results 결과가 더 좋은 경우 과감하게 더 나은 결과를 이용하세요.
            - research_results를 통해 elements 내용을 더 전략적이고 신뢰성 높게 수정해주세요. 
            - 꼭, elements와 슬라이드의 description과 관계된 데이터를 사용해주세요. ex). AI 자동화 솔루션 시장 규모 내용이 필요한데 AI 학습 데이터 시장을 찾은 경우 (제외)
            - elements 내용은 최종 ppt 내용에 사용이 되므로 최대한 자세하고 명확하게 작성이 필요하지만 필요없는 경우 제거해야합니다.(필요시 내용을 추가 및 제거)
            - 꼭, elements내 title은 슬라이드는 내용을 한눈에 알수 있게 간략하게 작성해주세요. ex). 글로벌 리테일 시장 개요
            - 꼭, middle_text(부제목)는 각 슬라이드의 title 추가 설명 및 슬라이드의 중요한 점을 한눈에 할수 있도록 자세히 작성해주세요.
            - elements 표, 그래프는 아래 형식에 따라 적용해주세요. (표, 그래프 내 데이터는 제안서에 가장 적합하게 변경해도 되며 research_results를 사용하도록 해주세요.)
            - TEMPLATE의 requirement_detail은 client_needs_summary에서 NeedsBulletPoints의 제안서 요구사항을 각각 설명하는 슬라이드로 사용할 예정으로 NeedsBulletPoints 별로 requirement_detail를 생성해주세요.
                ex). 
                client_needs_summary의 NeedsBulletPoints가 아래와 같다면 requirement_detail은 동일하게 각각에 대한 요구사항을 상세 설명해야합니다.
                선진사 및 트렌드 기반 필수 영역 제안
                글로벌 가이드라인 Framework 개발
                프로젝트 관리 방안 제시
            - 전체 PPT는 결국 requirement_detail에서 고객에게 우리가 어떻게 제안 할지를 나타내야합니다.
            - slide 순서는 제안서에 따라 자연스러운 흐름이 되도록 구성해주세요. ex). 고객의 니즈 → 시장환경 → 대응전략 → 실행계획 → 기대효과
            표 형식 예시:
            "KPIProjection": {{
                "description": "성과 지표 기반 기대 성과",
                "chart_type": "Bar Chart",
                "data": [["항목", "현재", "도입 후"], ["처리 시간", "5일", "2일"], ["운영 비용", "100%", "80%"]]
                "summary": "처리 시간이 60% 감소하였고 운영비용 20% 감소를 통해 KPI 개선이 되었음"
            }}
            
            그래프 형식 예시:
            "GraphLeft": {{
                "description": "시장 성장률",
                "graph_type": "Line Chart",
                "data_source": "출처: KOTRA, 2024",
                "data_table": [["연도", "시장 규모"], ["2023", "200억"], ["2024", "250억"]]
                "summary": "1년간 약 25% 증가하여 성장률이 매우 좋은 시장임"
            }}
            
            조건:
            - 반드시 Python 딕셔너리 형식 하나만 반환
            - 마크다운, 인용부호, 코드블록 금지
            - 한글로 작성
            - 제안서 내 필요없는 elements는 삭제 가능
            """

            response = openai.chat.completions.create(
                model="gpt-4o",
                messages=[{"role": "user", "content": prompt.strip()}],
                temperature=0.3
            )

            result_text = response.choices[0].message.content.strip()
            dict_text = extract_dict_from_gpt_response(result_text)

            parsed = ast.literal_eval(dict_text)

            refined_slides[slide_id]["slide_description"] = parsed.get("description", "nan")
            refined_slides[slide_id]["elements"] = parsed.get("elements", {})

        except Exception as e:
            print(f"[고도화 실패] {slide_id}: {e}")
            refined_slides[slide_id]["slide_description"] = "nan"
            refined_slides[slide_id]["elements"] = {}
            # 디버깅용 GPT 응답 출력 (옵션)
            # print(f"[응답 원문] {result_text}")

    return refined_slides

In [45]:
# 최종 ppt 초안 dictionary 생성
refined = refine_slides_dict(
    slides_dict=slides_dict_updated,
    rfp_text=rfp_text,
    user_inputs=user_inputs,
)

In [46]:
refined

{'cover_page': {'elements': {'Title': '삼성전자 MX 미국 직영 매장 PMO 프로젝트',
   'Subtitle': 'EY 컨설팅',
   'ProjectDate': '2025년 6월 25일',
   'PreparedBy': 'EY 컨설팅',
   'Logo': '삼성전자 및 EY 로고'},
  'needs_research': [],
  'slide_description': 'nan'},
 'table_of_contents': {'slide_description': 'nan',
  'description': '',
  'elements': {'Title': '목차',
   'SectionList': ['Executive Summary',
    'Project Understanding',
    'Client Needs Summary',
    'Requirement Details',
    'Market Analysis & Trends',
    'Strategic Recommendations',
    'Implementation Plan',
    'Risk Management Plan',
    'Expected Benefits',
    'Investment & Budget Estimation',
    'Team Introduction',
    'Why Us & Differentiation',
    'Closing Summary',
    'Q&A',
    'Appendix']},
  'needs_research': []},
 'executive_summary': {'slide_description': 'nan',
  'description': '삼성전자의 미국 직영 매장 구축 및 운영을 위한 프로젝트는 제조업과는 다른 리테일 분야의 새로운 도전입니다. EY 컨설팅은 선진사 벤치마크와 트렌드 분석을 통해 성공적인 매장 운영 모델을 제안합니다. 본 프로젝트는 4개월 간의 계약 기간 동안 매장 운영 전 영역의 매뉴얼화

In [47]:
import json
from datetime import datetime
# 오늘 날짜를 포함한 파일명 생성
today_str = datetime.now().strftime("%Y%m%d_%H%M")
file_name = f"DB/proposal/json/삼성전자_proposal_v1{today_str}.json"

# JSON 파일로 저장
with open(file_name, "w", encoding="utf-8") as f:
    json.dump(slides_dict, f, indent=2, ensure_ascii=False)

print(f"✅ JSON 저장 완료: {file_name}")


✅ JSON 저장 완료: DB/proposal/json/삼성전자_proposal_v120250627_0702.json
